<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<style>
    h1 {
        line-height: 200%;
        font-family: 'Comic Sans MS', cursive, sans-serif;
        color: #ffb6c1; /* رنگ نارنجی پاستیلی */
    }
</style>
</head>
<body>
    <h1>Zahra Alinaghipour</h1>
</body>
</html>


Analysis of the dataset of user searches on the MrTicket website.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

|Column|Description|
|:------:|:---:|
|`ServiceType`|Type of service|
|`TypedStrings`|List of user-typed strings in chronological order|
|`AcceptString`|String ultimately chosen by the user|

|Column|Description|
|:------:|:---:|
|`City EN`|City name in English|
|`City FA`|City name in Persian|
|`Province`|Province name|
|`Countise`|County name|
|`District`|District name|
|`Latitude`|Geographical latitude|
|`Longitude`|Geographical longitude|
|`Area`|Area in square kilometers|
|`Elevation`|Elevation in meters|
|`2016 Census`|Population in 2016|
|`2011 Census`|Population in 2011|
|`Wikipedia EN`|Link to English Wikipedia|
|`Wikipedia FA`|Link to Persian Wikipedia|
|`GeoHack`|GeoHack website link|

In [2]:
data = pd.read_json('../data/mrbilit_search.json')
data.head()

,ServiceType,TypedStrings,AcceptString
0,flight,"[, ف, فل, فلو]",فلورانس
1,hotel,"[, H, H,, H, , آ, آو, آوا, آواپ, آواپل, آواپلا...",شیراز
2,hotel,"[, آ, آو, آوا, آواپ, آواپل, آواپلا, آواپلاس]",آواپلاس
3,hotel,[],بین المللی اسپیناس خلیج فارس
4,bus,[],تهران


In [3]:
cities = pd.read_csv('../data/iran_cities.csv')
cities.head()

,City EN,City FA,Province,Countise,District,Latitude,Longitude,Area,Elevation,2016 Census,2011 Census,Wikipedia EN,Wikipedia FA,GeoHack
0,Ab Bar,آب‌بر,زنجان,طارم,مرکزی,36.918100,48.964700,709.0,NaN,8091.0,6725.0,https://en.wikipedia.org/wiki/Ab_Bar,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%E2%...,https://geohack.toolforge.org/geohack.php?lang...
1,Ab Pakhsh,آب‌پخش,بوشهر,دشتستان,آب‌پخش,29.363706,51.070180,251204.0,7003.0,18913.0,17238.0,https://en.wikipedia.org/wiki/Ab_Pakhsh,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%E2%...,https://geohack.toolforge.org/geohack.php?lang...
2,Abad,آباد,بوشهر,تنگستان,مرکزی,29.028300,51.249100,45.0,NaN,3787.0,3787.0,"https://en.wikipedia.org/wiki/Abad,_Bushehr",https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...
3,Abadan,آبادان,خوزستان,آبادان,مرکزی,30.339167,48.304167,3.0,NaN,231476.0,212744.0,"https://en.wikipedia.org/wiki/Abadan,_Iran",https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...
4,Abadeh,آباده,فارس,آباده,مرکزی,31.180000,52.670000,1890.0,NaN,59116.0,55758.0,https://en.wikipedia.org/wiki/Abadeh,https://fa.wikipedia.org/wiki/%D8%A2%D8%A8%D8%...,https://geohack.toolforge.org/geohack.php?lang...


Analyzing the popularity percentage of each service (bus, airplane, taxi, etc.).

In [4]:
percentages = {'bus': 0,
               'flight': 0,
               'train': 0,
               'hotel': 0,
               'taxi': 0,
               'ship': 0}

In [5]:
percentages = data['ServiceType'].value_counts(normalize=True).to_dict()
percentages

{'bus': 0.40237178034594756,
 'flight': 0.24959589092831785,
 'train': 0.17912229020318754,
 'hotel': 0.10012840849006722,
 'taxi': 0.05196767127426543,
 'ship': 0.016813958758214367}

In [6]:
fig = px.pie(values=percentages.values(),
             names=list(percentages.keys()),
             title='Pie Chart of Service Types')
fig.show()

In [7]:
preprocessed_data = data.copy()

In [8]:
preprocessed_data[preprocessed_data['AcceptString'].str.contains('- پایانه')]

,ServiceType,TypedStrings,AcceptString
225,bus,"[, ش, شي, شير, شيرا]",شیراز - پایانه امیرکبیر
878,bus,"[, , , ت, ته, تهر, تهرا, تهران]",تهران - پایانه جنوب
1003,bus,"[, شی, شیرا]",شیراز - پایانه کاراندیش
1031,bus,[],تهران - پایانه جنوب
1178,bus,"[ت, ته]",تهران - پایانه بیهقی
...,...,...,...
65919,bus,"[, ص, صغ, ص]",اصفهان - پایانه صفه
65929,bus,[],شیراز - پایانه کاراندیش
65971,bus,[],شیراز - پایانه کاراندیش
66175,bus,[],اصفهان - پایانه صفه


In [9]:
def get_city_name(x):
    # extract the city name
    city_name = x.split('-')[0]
    # remove whitespace
    city_name = ' '.join(city_name.split())
    return city_name

matched = preprocessed_data.loc[preprocessed_data['AcceptString'].str.contains('- پایانه'), 'AcceptString']
preprocessed_data.loc[preprocessed_data['AcceptString'].str.contains('- پایانه'), 'AcceptString'] = matched.transform(lambda x: get_city_name(x))

In [10]:
preprocessed_data[preprocessed_data['AcceptString'].str.contains('- پایانه')]

,ServiceType,TypedStrings,AcceptString


Most searches

In [11]:
data_transport = preprocessed_data[data['ServiceType'] != 'hotel']

In [12]:
top_cities = data_transport['AcceptString'].value_counts()[:20].index.tolist()
top_cities

['تهران',
 'مشهد',
 'اصفهان',
 'شیراز',
 'یزد',
 'بندرعباس',
 'اهواز',
 'قم',
 'تبریز',
 'کرمان',
 'رشت',
 'کرج',
 'ساری',
 'کیش',
 'کرمانشاه',
 'گرگان',
 'زاهدان',
 'زنجان',
 'اراک',
 'آبادان']

In [13]:
temp = data_transport[data_transport['AcceptString'].isin(top_cities)]
fig = px.histogram(temp, x="AcceptString", title='Histogram of Top 20 Cities')
fig.show()

Provinces with the most searches

In [14]:
merged_data = pd.merge(data_transport, cities, left_on='AcceptString', right_on='City FA', how='inner')
merged_data.head()

,ServiceType,TypedStrings,AcceptString,City EN,City FA,Province,Countise,District,Latitude,Longitude,Area,Elevation,2016 Census,2011 Census,Wikipedia EN,Wikipedia FA,GeoHack
0,bus,[],تهران,Tehran,تهران,تهران,تهران,مرکزی,35.7,51.4,90018004.0,7513.0,8693706.0,8154051.0,https://en.wikipedia.org/wiki/Tehran,https://fa.wikipedia.org/wiki/%D8%AA%D9%87%D8%...,https://geohack.toolforge.org/geohack.php?lang...
1,bus,[تهران],تهران,Tehran,تهران,تهران,تهران,مرکزی,35.7,51.4,90018004.0,7513.0,8693706.0,8154051.0,https://en.wikipedia.org/wiki/Tehran,https://fa.wikipedia.org/wiki/%D8%AA%D9%87%D8%...,https://geohack.toolforge.org/geohack.php?lang...
2,bus,[تهران],تهران,Tehran,تهران,تهران,تهران,مرکزی,35.7,51.4,90018004.0,7513.0,8693706.0,8154051.0,https://en.wikipedia.org/wiki/Tehran,https://fa.wikipedia.org/wiki/%D8%AA%D9%87%D8%...,https://geohack.toolforge.org/geohack.php?lang...
3,taxi,[تهران],تهران,Tehran,تهران,تهران,تهران,مرکزی,35.7,51.4,90018004.0,7513.0,8693706.0,8154051.0,https://en.wikipedia.org/wiki/Tehran,https://fa.wikipedia.org/wiki/%D8%AA%D9%87%D8%...,https://geohack.toolforge.org/geohack.php?lang...
4,bus,[تهران],تهران,Tehran,تهران,تهران,تهران,مرکزی,35.7,51.4,90018004.0,7513.0,8693706.0,8154051.0,https://en.wikipedia.org/wiki/Tehran,https://fa.wikipedia.org/wiki/%D8%AA%D9%87%D8%...,https://geohack.toolforge.org/geohack.php?lang...


In [15]:
counted = merged_data.groupby('Province')['AcceptString'].count().sort_values(ascending=False)
counted

Province
تهران                  12512
خراسان رضوی             6021
خوزستان                 3547
فارس                    3533
هرمزگان                 3322
مازندران                2364
یزد                     2241
کرمان                   2076
آذربایجان شرقی          2001
قم                      1711
گیلان                   1576
سیستان و بلوچستان       1323
سمنان                   1078
البرز                    952
بوشهر                    892
گلستان                   822
کرمانشاه                 747
آذربایجان غربی           676
اصفهان                   671
همدان                    650
مرکزی                    646
زنجان                    621
لرستان                   589
قزوین                    455
خراسان جنوبی             406
چهارمحال و بختیاری       297
ایلام                    268
اردبیل                   232
کردستان                  221
خراسان شمالی             132
کهگیلویه و بویراحمد       97
Name: AcceptString, dtype: int64

In [17]:
top_provinces = counted.index.tolist()[:15]
top_provinces

['تهران',
 'خراسان رضوی',
 'خوزستان',
 'فارس',
 'هرمزگان',
 'مازندران',
 'یزد',
 'کرمان',
 'آذربایجان شرقی',
 'قم',
 'گیلان',
 'سیستان و بلوچستان',
 'سمنان',
 'البرز',
 'بوشهر']

In [18]:
fig = px.histogram(merged_data, x="Province", title='Histogram of Most Searched Provinces')
fig.show()

Analysis of population:

Among cities with a population exceeding <b>500,000</b> according to the 2016 Census (<code dir=ltr>2016 Census</code> column), which cities are not among the top 20 searched cities (<code>top_cities</code> variable)?

In [19]:
not_in_top = []
for city in merged_data[merged_data['2016 Census']>500000]['AcceptString'].unique():
    if city not in top_cities:
        not_in_top.append(city)
print(not_in_top)

['اردبیل', 'ارومیه', 'همدان']
